In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.interpolate import griddata
from PSO import *
from MOO_func import *

In [2]:
def SNR_for_diff_para_mod(paras,f,h,PSD,figure_file=None,**kwargs):
    """ 
    Return 2D array of SNR for different parameters

    Parameters:
    -----------
    Armlength: L
    L1: float
        lower bound for L
    L2: float
        upper bound for L
    dL: float
        step size for L
    wavelength: l
    l1: float
        lower bound for l
    l2: float            
        upper bound for l    
    dl: float
        step size for l
    """
    L=paras[0]
    l=paras[1]
    
    df=f[1]-f[0]
   
    PSD_seq=PSD(f,L,l)
    SNRtemp = inner_prod(h,h,PSD_seq,df)
    SNR=np.sqrt(SNRtemp)
            
            
  
    return SNR

In [3]:
q=1e-4#mass ratio
    
    
# parameters
M_total= 10**5
mu=M_total*q
p0 = 12.0
e0 = 0.4
theta = np.pi/3  # polar viewing angle
phi = np.pi/4  # azimuthal viewing angle
dt = 10
dist=3
T=0.5
h = few(M_total, mu, p0, e0, theta, phi,dist=dist, dt=dt, T=T)  
h=h.get()

wave1 = np.array(h)
f = np.array(np.arange(len(h))/dt/ len(h))  
#tukey_seq=np.array([tukey(i,len(h),1/8) for i in range(0,len(h))])
#print(len(wave1),len(tukey_seq))
wave1 *= tukey(len(h), 1/8)
waveform1 = fft(wave1)
waveform2 = np.column_stack((waveform1, f))
temp=waveform2.real*waveform2.real+waveform2.imag*waveform2.imag
waveform = np.sqrt(temp)

fseq=waveform[1:len(waveform[:,1]),1]
h_f_SNR_EMRI=waveform[1:len(waveform[:,0]),0]
df_SNR_EMRI=waveform[1,1]-waveform[0,1]


In [4]:
def crcbpso_test_func_mod(x,params):
    

    if len(x.shape) == 1:
        x=x.reshape(1,-1)
        n_row = 1
        fit_val=np.zeros(n_row)
        valid_pts=np.ones(n_row)
        valid_pts=crcb_chk_std_srch_rng(x)
        fit_val[~valid_pts]=np.inf
        x[valid_pts]=s2rv(x[valid_pts],params)

        for lpc in range(0,n_row):
            if valid_pts[lpc]:
                x_temp=x[lpc]
                fit_val[lpc]=-SNR_for_diff_para_mod(x_temp,fseq,h_f_SNR_EMRI,PSD_L_lambda,figure_file=None)
    else:
        n_row, _ = x.shape
    
        fit_val=np.zeros(n_row)
        valid_pts=np.ones(n_row)
        valid_pts=crcb_chk_std_srch_rng(x)
        fit_val[~valid_pts]=np.inf

        x[valid_pts,:]=s2rv(x[valid_pts,:],params)

        for lpc in range(0,n_row):
            if valid_pts[lpc]:
                x_temp=x[lpc,:]
                fit_val[lpc]=-SNR_for_diff_para_mod(x_temp,fseq,h_f_SNR_EMRI,PSD_L_lambda,figure_file=None)
    return fit_val,x,r2sv(x,params)

: 

In [ ]:
#Test crcbpso
ffparams = {
   'r_min':np.array([1.0*10**8,200.0]),
   'r_max':np.array([10.0*10**9,1200.0]) }
#Fitness function handle
fit_func_handle=lambda x: crcbpso_test_func_mod(x,ffparams) 
#Call PSO
#np.random.seed(0)#Ensure that the random seed is correct
pso_out=crcbpso(fit_func_handle,2)
#Estimated parameters
std_coord=pso_out['best_location']
_,real_coord,_=fit_func_handle(std_coord)
print(f"Best location: {real_coord}")
print(f"Best fitness: {pso_out['best_fitness']}")

Step: 0 Best fitness -2.478970321212536
Step: 1 Best fitness -2.478970321212536
Step: 2 Best fitness -2.478970321212536
Step: 3 Best fitness -2.5688330008324973
Step: 4 Best fitness -2.579910110164993
Step: 5 Best fitness -2.6953166202493803
Step: 6 Best fitness -2.6953166202493803
Step: 7 Best fitness -2.6953166202493803
Step: 8 Best fitness -2.6953166202493803
Step: 9 Best fitness -2.6953166202493803
Step: 10 Best fitness -2.6953166202493803
Step: 11 Best fitness -2.6953166202493803
Step: 12 Best fitness -2.6953166202493803
Step: 13 Best fitness -2.6953166202493803
Step: 14 Best fitness -2.6953166202493803
Step: 15 Best fitness -2.703146248268277
Step: 16 Best fitness -2.703146248268277
Step: 17 Best fitness -2.703146248268277
Step: 18 Best fitness -2.703146248268277
Step: 19 Best fitness -2.703146248268277
Step: 20 Best fitness -2.703146248268277
Step: 21 Best fitness -2.703146248268277
Step: 22 Best fitness -2.703146248268277
Step: 23 Best fitness -2.703146248268277
Step: 24 Best f